# Experiment Parameters
Calculate the sample size and duration of marketing experiments.

### Fishers Exact Test

Used to compare the means of continuous variables. 

Used when the sample sizes are small. 

In [8]:
import numpy as np
from scipy.stats import fisher_exact

# Function to simulate Fisher's exact test for each sample size
def fisher_exact_power_simulation(alpha, power, p1, p2, max_sample_size=150000, iterations=100):
    for n in range(10, max_sample_size + 1):
        successes_control = np.random.binomial(n, p1, iterations)
        successes_variant = np.random.binomial(n, p2, iterations)
        significant_results = 0

        for control_success, variant_success in zip(successes_control, successes_variant):
            table = [[control_success, n - control_success],
                     [variant_success, n - variant_success]]
            _, p_value = fisher_exact(table)
            if p_value < alpha:
                significant_results += 1

        # Calculate the observed power
        observed_power = significant_results / iterations

        if observed_power >= power:
            return n
        
    return None

# Example usage
alpha = 0.05
power = 0.8
baseline_conversion = 0.0012
variant_conversion = 0.0024
iterations = 10  # Number of simulations per sample size

sample_size_per_group = fisher_exact_power_simulation(alpha, power, baseline_conversion, variant_conversion, iterations=iterations)
total_sample_size = sample_size_per_group * 2
daily_traffic = (1621+1621/30.5)
duration = total_sample_size / daily_traffic

print(f"Required sample size per group: {sample_size_per_group}")
print(f"Total sample size (both groups): {total_sample_size}")
print(f"Estimated test duration (days): {duration:.2f}")


Required sample size per group: 6506
Total sample size (both groups): 13012
Estimated test duration (days): 7.77


### Two Sided T Test
Used to compare the means of continuous variables (e.g., average time spent on the page). 

Used when the variance is unknown, data distribution deviates from normality, and sample sizes are small. 

Two sample is used if the metric can move in either direction. 

In [7]:
import math
from scipy.stats import t

# Function to calculate sample size for A/B test using t-test
def sample_size_t_test(baseline_ctr, mde, std_dev=None, alpha=0.05, power=0.8):
    # If standard deviation is not provided, approximate it based on baseline CTR
    if std_dev is None:
        std_dev = math.sqrt(baseline_ctr * (1 - baseline_ctr))

    # Degrees of freedom approximation for large samples
    df = 1000000  # Approximation to infinite degrees of freedom for t-distribution
    
    # T-scores for significance level and power
    t_alpha = t.ppf(1 - alpha / 2, df)  # Two-tailed test
    t_beta = t.ppf(power, df)
    
    # Standard error of the mean difference
    se = std_dev * math.sqrt(2)  # for two independent samples

    # Formula for sample size calculation
    sample_size_per_group = ((t_alpha + t_beta) ** 2 * se ** 2) / (mde ** 2)

    return math.ceil(sample_size_per_group)

# Function to calculate the test duration
def test_duration(sample_size, daily_traffic):
    return sample_size / daily_traffic

# Parameters
baseline_ctr = 0.05  # Example: 5% current CTR
mde = 0.01           # Example: Looking to detect a 1% change in CTR
alpha = 0.05         # Significance level (5%)
power = 0.8          # Power (80%)
daily_traffic = 1000 # Example: 1000 visitors per day

# Calculate sample size
sample_size = sample_size_t_test(baseline_ctr, mde, alpha=alpha, power=power)
total_sample_size = 2 * sample_size  # A/B test has two groups

# Calculate test duration
duration = test_duration(total_sample_size, daily_traffic)

print(f"Required sample size per group: {sample_size}")
print(f"Total sample size for the test: {total_sample_size}")
print(f"Test duration in days: {duration:.2f}")



Required sample size per group: 6280
Total traffic needed per group: 476812
Duration needed (days): 69


### Two Sided Z Test

To compare the means of a continuous variable or two proportions. 

Used when variance is known, data distribution is normal, and sample sizes are large. 

Two sided when the metric can move in either direction. 

In [ ]:
import math
from scipy.stats import norm

# Function to calculate sample size for A/B test
def sample_size_calculator(baseline_metric, mde, alpha=0.05, power=0.8):
    # Convert percentages to proportions
    p1 = baseline_metric
    p2 = baseline_metric + mde
    
    if p1 > 0 and p1 < 1 and p2 > 0 and p2 < 1 and alpha > 0 and alpha < 1 and power > 0 and power < 1:
        # Z-scores for significance level and power
        z_alpha = norm.ppf(1 - alpha / 2)  # Two-tailed test
        z_beta = norm.ppf(power)
        
        # Pooled proportion (average of the two conversion rates)
        pooled_prob = (p1 + p2) / 2
        
        # Standard error calculation
        se = math.sqrt(2 * pooled_prob * (1 - pooled_prob))
        
        # Formula for sample size calculation
        sample_size_per_group = ((z_alpha + z_beta) ** 2 * se ** 2) / (mde ** 2)
        
        return math.ceil(sample_size_per_group)
    else:
        print("Invalid input values. Check that all values are between 0 and 1.")

# Function to calculate test duration
def test_duration(sample_size, daily_traffic):
    return sample_size / daily_traffic

# Parameters
baseline_metric = 0.32  # Example: 5% current CTR
mde = 0.032           # Example: Looking to detect a 1% change
alpha = 0.05         # Significance level (5%)
power = 0.8          # Power (80%)
daily_traffic = (200000/30.5)  # Example: 1000 visitors per day

# Calculate sample size
sample_size = sample_size_calculator(baseline_metric, mde, alpha, power)
total_sample_size = 2 * sample_size  # A/B test has two groups

# Calculate test duration
duration = test_duration(total_sample_size, daily_traffic)

print(f"Required sample size per group: {sample_size}")
print(f"Total sample size for the test: {total_sample_size}")
print(f"Duration needed (days): {duration:.2f}")



Required sample size per group: 3421
Total sample size for the test: 6842
Duration needed (days): 1.04


### Chi Squared Test of Independence - Sample Size and Duration
Used to compare two samples of the same categorical variable to see if there is a significant difference or to compare two proportions. 

In [2]:
import numpy as np
from scipy.stats import norm

def calculate_sample_size(alpha, power, p1, p2):
    z_alpha = norm.ppf(1 - alpha / 2)
    z_beta = norm.ppf(power)
    n = ((z_alpha + z_beta) ** 2 * (p1 * (1 - p1) + p2 * (1 - p2))) / ((p1 - p2) ** 2)
    return np.ceil(n)

# Example usage
alpha = 0.05
power = 0.8
baseline_conversion = 0.32
variant_conversion = 0.352

sample_size = calculate_sample_size(alpha, power, baseline_conversion, variant_conversion)
total_sample_size = sample_size * 2
daily_traffic = (200000/30.5)
duration = total_sample_size / daily_traffic

print(f"Required sample size per group: {sample_size}")
print(f"Total sample size (both groups): {total_sample_size}")
print(f"Estimated test duration (days): {duration:.2f}")



Required sample size per group: 3417.0
Total sample size (both groups): 6834.0
Estimated test duration (days): 1.04
